In [1]:
import os
import pandas as pd
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../arch-dev-agent-87f23e12bec3.json"
PROJECT_ID = "arch-dev-agent" 
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

print("Conectado a Vertex AI")

Conectado a Vertex AI


In [ ]:
# esto iria en chunks.csv
# información REAL de tu negocio.
raw_data = [
    {
        "categoria": "políticas_devolución",
        "texto": "Nuestra política de devolución permite cambios hasta 30 días después de la compra. El producto debe estar sin uso, con etiquetas y en su caja original. No aceptamos devoluciones de ropa interior o calcetines abiertos."
    },
    {
        "categoria": "envíos",
        "texto": "Realizamos envíos a todo el Ecuador mediante Servientrega. El costo estándar es de $5.00 USD. El tiempo de entrega es de 24 a 48 horas laborables en ciudades principales y hasta 72 horas en trayectos especiales."
    },
    {
        "categoria": "pagos",
        "texto": "Aceptamos pagos mediante transferencia bancaria (Banco Pichincha, Guayaquil), tarjetas de crédito Visa/Mastercard a través de la app, y pagos contra entrega solo en la ciudad de Cuenca."
    },
    {
        "categoria": "garantía",
        "texto": "Todos nuestros zapatos tienen garantía de fábrica de 3 meses por defectos de pegado o costura. La garantía no cubre desgaste natural por uso en superficies abrasivas."
    }
]

# convertir a objetos Document
docs = [
    Document(
        page_content=item["texto"], 
        metadata={"category": item["categoria"]}
    ) 
    for item in raw_data
]

print(f"Datos preparados: {len(docs)} documentos de políticas.")

Datos preparados: 4 documentos de políticas.


In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

# generar embeddings
embeddings = VertexAIEmbeddings(
    model_name="text-embedding-004",
    project=PROJECT_ID
)

/tmp/ipykernel_68850/2373327122.py:5: DeprecationWarning: Use [`GoogleGenerativeAIEmbeddings`][langchain_google_genai.GoogleGenerativeAIEmbeddings] instead.
  embeddings = VertexAIEmbeddings(
/tmp/ipykernel_68850/2373327122.py:5: LangChainDeprecationWarning: The class `VertexAIEmbeddings` was deprecated in LangChain 3.2.0 and will be removed in 4.0.0. An updated version of the class exists in the `langchain-google-genai package and should be used instead. To use it run `pip install -U `langchain-google-genai` and import as `from `langchain_google_genai import GoogleGenerativeAIEmbeddings``.
  embeddings = VertexAIEmbeddings(


In [ ]:
# crear base de datos vectorial
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="sales_memory_v1"
)

print("Base de datos vectorial creada exitosamente.")

Base de datos vectorial creada exitosamente.


In [ ]:
# funcion de recuperacion RAG
def consultar_politicas(pregunta_usuario):
    # Busca el fragmento más relevante en los manuales para responder la duda.
    
    print(f"Buscando respuesta para: '{pregunta_usuario}'...")
    
    # buscar los 2 fragmentos más parecidos (k=2)
    resultados = vector_db.similarity_search(pregunta_usuario, k=1)
    
    if resultados:
        mejor_match = resultados[0]
        return f"INFORMACIÓN OFICIAL ENCONTRADA:\n{mejor_match.page_content}"
    else:
        return "No encontré información específica en nuestros manuales."

In [ ]:
print("\n--- TEST DE MEMORIA ---")

# pregunta sobre devoluciones
resp1 = consultar_politicas("¿Qué pasa si los zapatos no me quedan? ¿Puedo devolverlos?")
print(f"RESPUESTA SISTEMA:\n{resp1}\n")

# pregunta sobre envíos
resp2 = consultar_politicas("¿Cuánto cuesta el envío a Quito?")
print(f"RESPUESTA SISTEMA:\n{resp2}\n")

# pregunta irrelevante (Prueba de robustez)
resp3 = consultar_politicas("¿Cómo hago una pizza?")
print(f"RESPUESTA SISTEMA:\n{resp3}\n")


--- TEST DE MEMORIA ---
Buscando respuesta para: '¿Qué pasa si los zapatos no me quedan? ¿Puedo devolverlos?'...
RESPUESTA SISTEMA:
INFORMACIÓN OFICIAL ENCONTRADA:
Nuestra política de devolución permite cambios hasta 30 días después de la compra. El producto debe estar sin uso, con etiquetas y en su caja original. No aceptamos devoluciones de ropa interior o calcetines abiertos.

Buscando respuesta para: '¿Cuánto cuesta el envío a Quito?'...
RESPUESTA SISTEMA:
INFORMACIÓN OFICIAL ENCONTRADA:
Realizamos envíos a todo el Ecuador mediante Servientrega. El costo estándar es de $5.00 USD. El tiempo de entrega es de 24 a 48 horas laborables en ciudades principales y hasta 72 horas en trayectos especiales.

Buscando respuesta para: '¿Cómo hago una pizza?'...
RESPUESTA SISTEMA:
INFORMACIÓN OFICIAL ENCONTRADA:
Aceptamos pagos mediante transferencia bancaria (Banco Pichincha, Guayaquil), tarjetas de crédito Visa/Mastercard a través de la app, y pagos contra entrega solo en la ciudad de Cuenca.
